# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [20]:
# Write your code below.
%load_ext dotenv
%dotenv


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [21]:
import dask.dataframe as dd
import dask.dataframe as dd
import random
import pandas as pd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [22]:
import os
import glob

# Write your code below.

PRICE_DATA = os.getenv("PRICE_DATA")
PRICE_DATA

'../../05_src/data/prices/'

In [23]:
parquet_files = glob.glob(os.path.join(PRICE_DATA, "**", "*.parquet"), recursive=True)

dd_px = dd.read_parquet(parquet_files)

In [24]:
print(parquet_files[:5])

['../../05_src/data/prices/IMV/IMV_2019/part.0.parquet', '../../05_src/data/prices/IMV/IMV_2019/part.1.parquet', '../../05_src/data/prices/IMV/IMV_2017/part.0.parquet', '../../05_src/data/prices/IMV/IMV_2017/part.1.parquet', '../../05_src/data/prices/IMV/IMV_2010/part.0.parquet']


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [25]:
# Write your code below.
dd_px.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'source',
       'ticker', 'Year'],
      dtype='object')

In [38]:
dd_feat = dd_px.groupby("ticker", group_keys=False).apply(
          lambda x: x.assign(
              Close_lag_1 = x["Close"].shift(1),
              Adj_close_1 = x['Adj Close'].shift(1),
              Returns = (x['Close']/ x['Close'].shift(1)) - 1,
              hi_lo_range = (x['High'] - x['Low'])
              )
)

/var/folders/wy/v92nv28111j7vk4ynsyxm2700000gn/T/ipykernel_9258/2816814334.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = dd_px.groupby("ticker", group_keys=False).apply(


In [39]:
dd_feat

,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag_1,Adj_close_1,Returns,hi_lo_range
npartitions=3218,,,,,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,float64,object,object,int32,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [40]:
# Write your code below.

# to transform a dask.dataframe to a pandas dataframe
pandas_dd_feat = dd_feat.compute()
pandas_dd_feat

,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag_1,Adj_close_1,Returns,hi_lo_range
579,2018-07-03,29.900000,30.430000,29.844999,30.059999,30.059999,43300.0,HCM.csv,HCM,2018,NaN,NaN,NaN,0.585001
580,2018-07-05,30.260000,30.639999,29.990000,30.400000,30.400000,33500.0,HCM.csv,HCM,2018,30.059999,30.059999,0.011311,0.650000
581,2018-07-06,30.309999,30.820000,29.711000,30.620001,30.620001,40900.0,HCM.csv,HCM,2018,30.400000,30.400000,0.007237,1.108999
582,2018-07-09,30.670000,30.879999,29.650000,29.719999,29.719999,225300.0,HCM.csv,HCM,2018,30.620001,30.620001,-0.029393,1.230000
583,2018-07-10,29.750000,30.879999,29.330000,30.100000,30.100000,192700.0,HCM.csv,HCM,2018,29.719999,29.719999,0.012786,1.549999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4873,2015-06-26,14.900000,14.980000,14.630000,14.700000,14.700000,258700.0,LNDC.csv,LNDC,2015,14.840000,14.840000,-0.009434,0.349999
4874,2015-06-29,14.670000,14.790000,14.380000,14.410000,14.410000,88900.0,LNDC.csv,LNDC,2015,14.700000,14.700000,-0.019728,0.410000
4875,2015-06-30,14.460000,14.540000,14.300000,14.430000,14.430000,59700.0,LNDC.csv,LNDC,2015,14.410000,14.410000,0.001388,0.240000
4876,2015-07-01,14.530000,14.610000,14.310000,14.420000,14.420000,33100.0,LNDC.csv,LNDC,2015,14.430000,14.430000,-0.000693,0.299999


In [37]:
pandas_dd_feat.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'source',
       'ticker', 'Year', 'Close_lag_1', 'Adj_close_1', 'Returns',
       'hi_lo_range'],
      dtype='object')

In [ ]:
group = pandas_dd_feat.groupby("ticker", group_keys=False).apply(
    lambda x: x.assign(avg_returns_V2=x["Returns"].rolling(10).mean())
)


/var/folders/wy/v92nv28111j7vk4ynsyxm2700000gn/T/ipykernel_9258/1240205902.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  group = pandas_dd_feat.groupby("ticker", group_keys=False).apply(


In [42]:
group

,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag_1,Adj_close_1,Returns,hi_lo_range,avg_returns_V2
579,1994-03-28,241.666672,241.666672,233.333328,240.000000,240.000000,1100.0,AMSC.csv,AMSC,1994,241.666672,241.666672,-0.006897,8.333344,0.009328
579,2007-01-11,18.580000,19.250000,18.510000,19.230000,19.230000,346500.0,BECN.csv,BECN,2007,18.490000,18.490000,0.040022,0.740000,-0.007595
579,2015-01-27,32.750000,32.980000,32.549999,32.689999,32.689999,1355400.0,BERY.csv,BERY,2015,33.060001,33.060001,-0.011192,0.430000,0.000230
579,2007-06-05,17.250000,17.400000,16.200001,16.700001,16.700001,93000.0,CBAT.csv,CBAT,2007,17.200001,17.200001,-0.029070,1.199999,0.004467
579,1982-07-01,2.481482,2.481482,2.481482,2.481482,0.221333,0.0,CPK.csv,CPK,1982,2.481482,0.221333,0.000000,0.000000,-0.007105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215433,2016-12-23,32.919998,33.000000,32.660000,32.900002,31.320290,4028100.0,AMAT.csv,AMAT,2016,33.009998,31.425011,-0.003332,0.340000,0.001757
215434,2016-12-27,33.150002,33.459999,33.099998,33.330002,31.729660,5008500.0,AMAT.csv,AMAT,2016,32.900002,31.320290,0.013070,0.360001,0.004146
215435,2016-12-28,33.549999,33.680000,32.880001,32.900002,31.320290,5388600.0,AMAT.csv,AMAT,2016,33.330002,31.729660,-0.012901,0.799999,0.002762
215436,2016-12-29,32.799999,32.990002,32.480000,32.660000,31.091822,4472500.0,AMAT.csv,AMAT,2016,32.900002,31.320290,-0.007295,0.510002,0.001690


In [ ]:
returns = pandas_dd_feat['Returns'].to_frame()
returns['average_returns'] = returns['Returns'].rolling(10).mean()

returns

,Returns,average_returns
579,NaN,NaN
580,0.011311,NaN
581,0.007237,NaN
582,-0.029393,NaN
583,0.012786,NaN
...,...,...
4873,-0.009434,0.001461
4874,-0.019728,-0.000028
4875,0.001388,0.000041
4876,-0.000693,0.000804


In [44]:
# Calculate Cumulative Moving Average (CMA) with a window of 30
returns['CMA10'] = returns['Returns'].expanding().mean()

# Print DataFrame
returns

,Returns,average_returns,CMA10
579,NaN,NaN,NaN
580,0.011311,NaN,0.011311
581,0.007237,NaN,0.009274
582,-0.029393,NaN,-0.003615
583,0.012786,NaN,0.000485
...,...,...,...
4873,-0.009434,0.001461,0.626342
4874,-0.019728,-0.000028,0.626340
4875,0.001388,0.000041,0.626338
4876,-0.000693,0.000804,0.626337


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

1. We convert the Dask objects into DataFrames for several reasons:
    - DataFrames allow us to handle large files in terms of memory.
    - The style of Pandas is much easier to understand.
    - For optimization of workflows, Dask requires explicit instructions on what should be done, while Pandas helps reduce unnecessary computations.

2. It is not strictly required to convert to Pandas, as we can perform any calculations on Dask DataFrames. It can simply manage large datasets without any issues, and it can lead to substantial performance improvements.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.